# ejercicio 5

Se asume que la distribucion subayecente $X$ a la muestra  es una binomial $Bi(8,p)$ con $p$ desconocido. Bajo esta suposicion sabesmos que $E[X]=np=8p$. Por lo tanto si estimamos la esperanza como la media muestral $E[X]\sim\overline X$ obtenemos:
$$
p\sim\frac{\overline X}{8}=\hat p
$$

Calcularemos luego el p-valor de la prueba de que los datos corresponden a la $Bi(8,\hat p)$ segun el test **chicuadrado** y por **simulacion**.

La binomial toma valores en $[0,\ldots,8]$ el $T$ estaditico utilizado queda:
$$
T = \sum_{i=0}^8 \frac{(N_i - Np_i)^2}{Np_i}
$$

Recordando que $N_i$ son las frecuencias de la muestra, $N$ el tamanio de la muestra y $Np_i$ las frecuencias teoricas.

Como se toman $k=9$ valores y se debe estimar **un** paramestro $(p)$ los grados de libertad de la variable chicuadrado para el test seran $(k-1)-1=7$.

N

In [ ]:
import numpy as np
from scipy.stats import chi2
from scipy.stats import binom

# Definimos T estadistico
def T(freq_obs,freq_teo):
  return np.sum((np.array(freq_obs) - np.array(freq_teo))**2 / np.array(freq_teo))

muestra = [6, 7, 3, 4, 7, 3, 7, 2, 6, 3, 7, 8, 2, 1, 3, 5, 8, 7]
n = len(muestra)

n_binom = 8
# estimamos p de la binomial en base a la muestra
p_estimate = np.mean(muestra)/n_binom

# construimois la frecuencia teorica
rv = binom(n_binom, p_estimate)
k_values = np.arange(n_binom + 1) # binomial(n_binom,p) toma valores [0,...,8]
freq_teo = rv.pmf(k_values) * n

# frecuencias observadas
freq_obs = np.bincount(muestra, minlength=n_binom+1)

#============= p-valor mediante el test chicuadrado ============================


k = len(freq_teo)
# grados de libertad = #(valores posibles) - 1 - #(parametros estimados)
#                    = k - 1 - 1
df = k - 2
# evaluamos el estadistico en frecuencias observadas vs teoricas
T_obs = T(freq_obs,freq_teo)
#calculamos el p-valor
p_val_chi = 1 - chi2.cdf(T_obs,df=df)


#============= p-valor mediante simulaciones ===================================

n_simulaciones = int(1e4)
np.random.seed(42)
T_simulados = [] # lista de T para cada simulacion
p_dif = [] # lista de p estimados para cada simulaicon

for i in range(n_simulaciones):
    # generar muestra binomial(8,p) del mismo tamanio que la original (n)
    # donde p es el estimado para la muestra original
    muestra_sim = binom.rvs(n_binom, p_estimate, size=n)

    # estimamos el p de la muestra generada
    p_estimate_sim = np.mean(muestra_sim)/n_binom
    p_dif.append(p_estimate_sim)

    # construimos la frecuencia teorica en base al nuevo p estimado
    rv = binom(n_binom, p_estimate_sim)
    k_values = np.arange(n_binom + 1)
    freq_teo_sim = rv.pmf(k_values) * n

    # contruimos frecuencias observadas de la nueva muestra
    freq_obs_sim = np.bincount(muestra_sim, minlength=n_binom+1)

    # evaluamos el estadistico en frecuencias observadas vs teoricas
    T_sim = T(freq_obs_sim,freq_teo_sim)
    T_simulados.append(T_sim)

# calculamos el p-valor de simulacion
T_simulados = np.array(T_simulados)
p_val_sim = np.mean(T_simulados >= T_obs)

#===============================================================================

print(f"test statistic: {T_obs:.4f}\n"
      f"{n_simulaciones} muestras de tamanio {n}\n" + "="*100 + "\n"
      f"p-value (chi-cuadrado, gardos de libertad {k-2}): {p_val_chi:.5f}\n"
      f"                        p-value (simulación): {p_val_sim:.5f}\n" + "="*100 + "\n"
      f"p estimado de la mustra - promedio de p estimado en cada simulacion: {np.abs(p_estimate - np.mean(p_dif)):.7f}")

test statistic: 31.4993
10000 muestras de tamanio 18
p-value (chi-cuadrado, gardos de libertad 7): 0.00005
                        p-value (simulación): 0.01010
p estimado de la mustra - promedio de p estimado en cada simulacion: 0.0001285


Notar las discrepacia entre los p-valores obtenidos segun cada metodo, no observada en ejercios anteriores.